# FloPy

### Demo of netCDF and shapefile export capabilities within the flopy export module. 

In [1]:
from __future__ import print_function
import os
import sys
import flopy

print(sys.version)
print('flopy version: {}'.format(flopy.__version__))

3.5.3 | packaged by conda-forge | (default, Feb 10 2017, 07:09:50) 
[GCC 4.2.1 Compatible Apple LLVM 7.3.0 (clang-703.0.31)]
flopy version: 3.2.7


Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("..", "data", "freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load(nam_file, model_ws=model_ws, check=False)


/Users/aleaf/anaconda/lib/python3.5/site-packages/flopy-3.2.7-py3.5.egg/flopy/utils/reference.py:352: UserWarning: rotation arg has recently changed. It was previously treated as positive clockwise. It now is positive counterclockwise.
  warnings.warn(msg)
/Users/aleaf/anaconda/lib/python3.5/site-packages/flopy-3.2.7-py3.5.egg/flopy/utils/reference.py:605: UserWarning: rotation arg has recently changed. It was previously treated as positive clockwise. It now is positive counterclockwise.
  warnings.warn(msg)
/Users/aleaf/anaconda/lib/python3.5/site-packages/flopy-3.2.7-py3.5.egg/flopy/utils/reference.py:465: UserWarning: rotation arg has recently changed. It was previously treated as positive clockwise. It now is positive counterclockwise.
  warnings.warn(msg)


We can see the ``SpatialReference`` instance has generic entries, as does ``start_datetime``

In [3]:
ml.dis.sr

xul:619653; yul:3353277; rotation:15; proj4_str:+proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs; units:meters; lenuni:2; length_multiplier:1.0

In [4]:
ml.dis.start_datetime

'1/1/2015'

Setting the attributes of the ``ml.dis.sr`` is easy:

In [5]:
ml.dis.sr.xul = 123456.7
ml.dis.sr.yul = 765432.1
rotation = 15.0
proj4_str = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
ml.dis.start_datetime = '7/4/1776'
ml.dis.sr

xul:123456.7; yul:765432.1; rotation:15; proj4_str:+proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs; units:meters; lenuni:2; length_multiplier:1.0

In [6]:
ml.dis.start_datetime

'7/4/1776'

### Some netCDF export capabilities:

#### Export the whole model (inputs and outputs)

In [7]:
# make directory
pth = os.path.join('data', 'netCDF_export')
if not os.path.exists(pth):
    os.makedirs(pth)

In [8]:
fnc = ml.export(os.path.join(pth, ml.name+'.in.nc'))
hds = flopy.utils.HeadFile(os.path.join(model_ws,"freyberg.hds"))
flopy.export.utils.output_helper(os.path.join(pth, ml.name+'.out.nc'), ml, {"hds":hds})

#### export a single array to netcdf or shapefile

In [9]:
# export a 2d array
ml.dis.top.export(os.path.join(pth, 'top.nc'))
ml.dis.top.export(os.path.join(pth, 'top.shp'))

wrote data/netCDF_export/top.shp


#### sparse export of stress period data for a boundary condition package  
* excludes cells that aren't in the package (aren't in `package.stress_period_data`)  
* by default, stress periods with duplicate parameter values (e.g., stage, conductance, etc.) are omitted
(`squeeze=True`); only stress periods with different values are exported  
* argue `squeeze=False` to export all stress periods

In [10]:
ml.drn.stress_period_data.export(os.path.join(pth, 'drn.shp'), sparse=True)

wrote data/netCDF_export/drn.shp


#### Export a 3d array

In [11]:
#export a 3d array
ml.upw.hk.export(os.path.join(pth, 'hk.nc'))
ml.upw.hk.export(os.path.join(pth, 'hk.shp'))

wrote data/netCDF_export/hk.shp


#### Export a number of things to the same netCDF file

In [12]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(pth, 'test.nc'))
ml.upw.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


### Export whole packages to a netCDF file

In [13]:
# export mflist
fnc = ml.wel.export(os.path.join(pth, 'packages.nc'))
ml.upw.export(fnc)
fnc.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.2.7
    date_created: 2017-06-20T22:01:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.83250045776
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 123456.7
    origin_y: 765432.1
    origin_crs: +proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs
    grid_rotation_from_origin: 15.0
    create_platform: Darwin
    create_directory: /Users/aleaf/Documents/GitHub/flopy3/examples/Notebooks
    flopy_sr_yul: 765432.1
    exe_name: mf2005.exe
    flopy_sr_rotation: 15.0
    solver_flux_tolerance: 500.0
    create_hostname: IGSARMEWLTATL-M
    modflow_version: mfnwt
    flopy_sr_proj4_str: +proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs
    flopy_sr_xul: 123456.7
    start_datetime: 7/4/1776
    model_ws: ../data/freyberg_multilayer_transient
    sol

### Export the whole model to a netCDF

In [14]:
fnc = ml.export(os.path.join(pth, 'model.nc'))
fnc.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.2.7
    date_created: 2017-06-20T22:01:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.83250045776
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 123456.7
    origin_y: 765432.1
    origin_crs: +proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs
    grid_rotation_from_origin: 15.0
    create_platform: Darwin
    create_directory: /Users/aleaf/Documents/GitHub/flopy3/examples/Notebooks
    flopy_sr_yul: 765432.1
    exe_name: mf2005.exe
    flopy_sr_rotation: 15.0
    solver_flux_tolerance: 500.0
    create_hostname: IGSARMEWLTATL-M
    modflow_version: mfnwt
    flopy_sr_proj4_str: +proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs
    flopy_sr_xul: 123456.7
    start_datetime: 7/4/1776
    model_ws: ../data/freyberg_multilayer_transient
    sol